In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from fairness.pre_processing import categorical_to_numeric_converter
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [12]:
dataset = pd.read_csv('./dataset/adult/adult.data', sep=',')

In [13]:
print("Columns of the dataset")

Columns of the dataset


In [14]:
dataset = categorical_to_numeric_converter(dataset)

In [15]:
output_column = "income"

In [16]:
X = dataset.iloc[:,:len(dataset.columns) - 2]
y = dataset.loc[:, output_column:]
y = np.ravel(y)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
standard_scaler_train = StandardScaler()
X_train = standard_scaler_train.fit_transform(X_train)

standard_scaler_test = StandardScaler()
X_test = standard_scaler_test.fit_transform(X_test)

In [19]:
random_forest_classifier = RandomForestClassifier()
decision_tree_classifier = DecisionTreeClassifier()
knn_classifier = KNeighborsClassifier()

models = [
    random_forest_classifier,
    decision_tree_classifier,
    knn_classifier
]

random_forest_params = {
    'min_samples_leaf': [5, 7, 9, 11],
    'n_estimators': [200, 500],
    'max_depth' : [10, 20, 50, 80, 100, 150],
    'criterion' :['gini', 'entropy']
}

decision_tree_params = {
    'min_samples_leaf': [5, 7, 9, 11],
    'max_depth' : [10, 20, 50, 80, 100, 150],
    'criterion' :['gini', 'entropy']
}

knn_params = {
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(20, 50),
}


model_params = [random_forest_params, decision_tree_params, knn_params]

In [22]:
best_params = None
max_accuracy = 0
best_model = None

for model_index in range(0, len(models)):
    
    grid_search = GridSearchCV(
    estimator=models[model_index],
    param_grid=model_params[model_index],
    scoring='accuracy', 
    return_train_score=True,
    cv=3
    )

    grid_search_classifier = grid_search.fit(X_train, y_train)
    y_pred = grid_search_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print("Accuracy: ", accuracy)
    print("Model: ", models[model_index])
    print("Best params: ", grid_search_classifier.best_params_)
    
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_params = grid_search_classifier.best_params_
        best_model = models[model_index]

Accuracy:  0.8573422668900056
Model:  RandomForestClassifier()
Best params:  {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 5, 'n_estimators': 500}
Accuracy:  0.8346361436813698
Model:  DecisionTreeClassifier()
Best params:  {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 11}
Accuracy:  0.831286059929276
Model:  KNeighborsClassifier()
Best params:  {'algorithm': 'auto', 'leaf_size': 20}


In [ ]:
print("BEST SOLUTION")
print("Best model: ", best_model)
print("Best_params: ", best_params)
print("Accuracy: ", max_accuracy)

Best model:  RandomForestClassifier()
Best_params:  {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 5, 'n_estimators': 200}
Accuracy:  0.8572492090080029


In [ ]:
#print("Accuracy score on Test set: ", accuracy_score(y_test, y_pred))
#print("Accuracy score on Train set: ", accuracy_score(y_train, grid_search_classifier.predict(X_train)))
#print(classification_report(y_test, y_pred))

In [ ]:
#cm_display = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=[False, True])
#cm_display.plot()
#plt.show()